In [7]:
import ollama

In [8]:
def classify_english_question(english_question):
    prompt = f"""You are given a question in English. Your task is to classify it as either SINGLEHOP or MULTIHOP.

Definitions:
- A question is **SINGLEHOP** if it can be answered using a single fact, sentence, or document. The question may be long, but if it doesn't require combining or reasoning over multiple distinct pieces of information, it's SINGLEHOP.
- A question is **MULTIHOP** if answering it requires combining multiple facts, reasoning over several steps, or connecting pieces of information from different sources.

⚠️ Important:
- A question's length does not determine its type. A long, descriptive question can still be SINGLEHOP.
- MULTIHOP questions typically require you to first find one piece of information, then use that to find the next.

Examples:

🔹 Example 1 (SINGLEHOP - factual entity lookup):
Question: "Who wrote Pride and Prejudice?"
Answer: SINGLEHOP

🔹 Example 2 (SINGLEHOP - simple date):
Question: "When did the Berlin Wall fall?"
Answer: SINGLEHOP

🔹 Example 3 (SINGLEHOP - location fact):
Question: "Where is the Eiffel Tower located?"
Answer: SINGLEHOP

🔹 Example 4 (SINGLEHOP - biography-style):
Question: "What is the profession of Elon Musk?"
Answer: SINGLEHOP

🔹 Example 5 (SINGLEHOP - short explanation):
Question: "What is photosynthesis?"
Answer: SINGLEHOP

🔸 Example 6 (MULTIHOP - indirect relationship):
Question: "Which country was ruled by the emperor who built the Taj Mahal?"
Answer: MULTIHOP

🔸 Example 7 (MULTIHOP - event inference):
Question: "Which city hosted the Olympics where Michael Phelps won 8 gold medals?"
Answer: MULTIHOP

🔸 Example 8 (MULTIHOP - entity resolution):
Question: "Who is the father of the current King of the United Kingdom?"
Answer: MULTIHOP

🔸 Example 9 (MULTIHOP - causal historical link):
Question: "What war led to the independence of the United States?"
Answer: MULTIHOP

🔸 Example 10 (MULTIHOP - layered facts):
Question: "Which university did the author of 'The Selfish Gene' attend, and what subject did he study there?"
Answer: MULTIHOP

---

Now read the following English question and classify it:

Question: "{english_question}"
Answer:"""

    try:
        response = ollama.generate(
            model='llama3:8b',
            prompt=prompt
        )
        reply = response['response'].strip().lower()

        # Normalize and return classification
        if 'multihop' in reply:
            return 'multihop'
        elif 'singlehop' in reply or 'simple' in reply:
            return 'singlehop'

        print(f"⚠️ Unexpected response: {reply}")
        return "unknown"

    except Exception as e:
        print(f"❌ Error processing question: {english_question}\n↪ {e}")
        return "error"
    





def classify_english_question_using_gemma(english_question):
    prompt = f"""You are given a question in English. Your task is to classify it as either SINGLEHOP or MULTIHOP.

Definitions:
- A question is **SINGLEHOP** if it can be answered using a single fact, sentence, or document. The question may be long, but if it doesn't require combining or reasoning over multiple distinct pieces of information, it's SINGLEHOP.
- A question is **MULTIHOP** if answering it requires combining multiple facts, reasoning over several steps, or connecting pieces of information from different sources.

⚠️ Important:
- A question's length does not determine its type. A long, descriptive question can still be SINGLEHOP.
- MULTIHOP questions typically require you to first find one piece of information, then use that to find the next.


---

Now read the following English question and classify it:

Question: "{english_question}"
Answer:"""

    try:
        response = ollama.generate(
            model='gemma:7b-instruct',
            prompt=prompt
        )
        reply = response['response'].strip().lower()

        # Normalize and return classification
        if 'multihop' in reply:
            return 'multihop'
        elif 'singlehop' in reply or 'simple' in reply:
            return 'singlehop'

        print(f"⚠️ Unexpected response: {reply}")
        return "unknown"

    except Exception as e:
        print(f"❌ Error processing question: {english_question}\n↪ {e}")
        return "error"



In [9]:
def decompose_english_query(english_query: str) -> dict:
    """Returns dictionary with q1 and q2 keys containing sub-questions"""
    refined_prompt = f"""
You are an expert in breaking down complex English questions into two logically ordered sub-questions.

Rules:
- Extract exactly 2 sub-questions.
- q1 should logically precede q2.
- Use clear and grammatically correct English.
- Output only in this format:
  q1: [First sub-question]
  q2: [Second sub-question]

Input: {english_query}
Output:
"""

    try:
        response = ollama.generate(
            model='llama3:8b',
            prompt=refined_prompt,
            options={
                'temperature': 0.5,
                'num_ctx': 2048  # Now you have more room for longer inputs
            }
        )

        output = response['response'].strip()

        result = {}
        for line in output.split('\n'):
            line = line.strip()
            if line.startswith('q1:'):
                result['q1'] = line[3:].strip()
            elif line.startswith('q2:'):
                result['q2'] = line[3:].strip()

        return result if len(result) == 2 else {}

    except Exception as e:
        print(f"Decomposition error: {str(e)}")
        return {}
    


def decompose_english_query_using_gemma(english_query: str) -> dict:
    """Returns dictionary with q1 and q2 keys containing sub-questions"""
    refined_prompt = f"""
You are an expert in breaking down complex English questions into two logically ordered sub-questions.

Rules:
- Extract exactly 2 sub-questions.
- q1 should logically precede q2.
- Use clear and grammatically correct English.
- Output only in this format:
  q1: [First sub-question]
  q2: [Second sub-question]

Input: {english_query}
Output:
"""

    try:
        response = ollama.generate(
            model='gemma:7b-instruct',
            prompt=refined_prompt,
            options={
                'temperature': 0.5,
                'num_ctx': 2048  # Now you have more room for longer inputs
            }
        )

        output = response['response'].strip()

        result = {}
        for line in output.split('\n'):
            line = line.strip()
            if line.startswith('q1:'):
                result['q1'] = line[3:].strip()
            elif line.startswith('q2:'):
                result['q2'] = line[3:].strip()

        return result if len(result) == 2 else {}

    except Exception as e:
        print(f"Decomposition error: {str(e)}")
        return {}



In [10]:
# Sample complex English query
sample_query = "What were the main causes of World War II, and how did they lead to the involvement of the United States?"

# Call the function
result = decompose_english_query_using_gemma(sample_query)

# Display the result
print("Decomposed sub-questions:")
print("Q1:", result.get('q1'))
print("Q2:", result.get('q2'))


Decomposed sub-questions:
Q1: What were the main causes of World War II?
Q2: How did these causes lead to the involvement of the United States in the war?


In [11]:
def query_context_relevance_check_en(query: str, context: str) -> bool:
    prompt = f"""
You are a binary classifier.

Your task is to decide whether the following *context* is relevant to the *question*. You must answer ONLY with **True** or **False** — no explanation, no commentary, just one word: True or False.

Criteria:
- If the context helps answer the question directly or indirectly, reply: True
- If the context is unrelated, confusing, or insufficient, reply: False

IMPORTANT:
- Do NOT explain your answer.
- Do NOT include any additional comments.
- Just respond with: True or False

---

Question: {query}

Context: {context}

Answer (True/False):
"""

    try:
        import ollama
        response = ollama.chat(
            model="llama3:8b",
            messages=[{"role": "user", "content": prompt}]
        )
        answer = response['message']['content'].strip().lower()
        return answer == 'true'
    except Exception as e:
        print(f"Error during relevance check: {e}")
        return False



def query_context_relevance_check_en_using_gemma(query: str, context: str) -> bool:
    prompt = f"""
You are a binary classifier.

Your task is to decide whether the following *context* is relevant to the *question*. You must answer ONLY with **True** or **False** — no explanation, no commentary, just one word: True or False.

Criteria:
- If the context helps answer the question directly or indirectly, reply: True
- If the context is unrelated, confusing, or insufficient, reply: False

IMPORTANT:
- Do NOT explain your answer.
- Do NOT include any additional comments.
- Just respond with: True or False

---

Question: {query}

Context: {context}

Answer (True/False):
"""

    try:
        import ollama
        response = ollama.chat(
            model="gemma:7b-instruct",
            messages=[{"role": "user", "content": prompt}]
        )
        answer = response['message']['content'].strip().lower()
        return answer == 'true'
    except Exception as e:
        print(f"Error during relevance check: {e}")
        return False


In [12]:
from sentence_transformers import SentenceTransformer
import faiss
import pickle
import torch

def load_retriever(
    index_path: str,
    chunks_path: str
):
    # Initialize device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Load SentenceTransformer MiniLM model (lighter than intfloat/e5-large)
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=device)
    
    # You can optionally configure max length
    model.max_seq_length = 512  # if needed for long sentences
    model.tokenizer.do_lower_case = False  # Keep for Urdu if using custom tokenizer

    # Load FAISS index
    index = faiss.read_index(index_path)
    
    # Load stored chunks
    with open(chunks_path, "rb") as f:
        chunks_list = pickle.load(f)
    
    return model, index, chunks_list, device


d:\Academic Work\Multihop_Project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
model, index, chunks_list, device = load_retriever(
    index_path="../../vector_db/paragraphs/5884_paras/5884_paras_index.faiss",
    chunks_path="../../data_storage/Paragraph_chunks/5884_paragraphs/5884_parachunks.pkl"
)

In [14]:
def retrieve_documents(query, k=3):
    # Encode the query using MiniLM model
    query_embedding = model.encode([query])
    
    # Search the FAISS index
    _, indices = index.search(query_embedding, k)
    
    # Return the top-k retrieved chunks
    return [chunks_list[i] for i in indices[0]]

In [15]:
def generate_using_llama3(context, query):
    prompt = f"""You are a helpful assistant designed to generate precise and informative answers based strictly on the given context.

Query:
{query}

Retrieved Context:
{context}

Instruction:
Answer the query using only the information present in the retrieved context. If the answer is not directly stated, make the best possible inference from the available context. Do not say "no information available", "cannot answer", or provide disclaimers. Only return a clear and direct answer — no introductions, no explanations, and no repetition of the query."""


    try:
        response = ollama.generate(
            model='llama3:8b',
            prompt=prompt
        )
        return response['response'].strip()
    except Exception as e:
        print("Error during generation:", e)
        return "Error generating answer."


In [16]:
def get_context_of_multihop_without_parallel(query,model=model,index=index,chunks_list=chunks_list,k=3):


    classification = classify_english_question_using_gemma(query)


    if classification == "singlehop":
        retrieved_context = retrieve_documents(query,k)
        return retrieved_context
        

    if classification == "multihop":
        decomposition = decompose_english_query_using_gemma(query)
        q1 = decomposition.get("q1", "")
        q2 = decomposition.get("q2", "")

        main_context = retrieve_documents(q1, k)

        for i in range(min(len(main_context), k)):
            intermediate_ctx = main_context[i]
            
            combined_query = q1 + intermediate_ctx + q2
            
            second_hop_contexts = retrieve_documents(combined_query, k)
            
            for ctx in second_hop_contexts:
                if query_context_relevance_check_en_using_gemma(query, ctx):
                    main_context.append(ctx)
        
        return main_context    
 


In [17]:

import time
from concurrent.futures import ThreadPoolExecutor, as_completed

def expand_multihop_context(intermediate_ctx, query, q1, q2, k):
    try:
        combined_query = q1 + intermediate_ctx + q2
        second_hop_contexts = retrieve_documents(combined_query, k)

        relevant_contexts = []

        with ThreadPoolExecutor() as inner_executor:
            futures = [
                inner_executor.submit(query_context_relevance_check_en_using_gemma, query, ctx)
                for ctx in second_hop_contexts
            ]

            for i, future in enumerate(as_completed(futures)):
                try:
                    if future.result():
                        relevant_contexts.append(second_hop_contexts[i])
                except Exception as e:
                    print("Error during relevance check:", e)

        return relevant_contexts

    except Exception as e:
        print("Error in expand_multihop_context:", e)
        return []


def get_context_of_multihop(query, type, model=model, index=index, chunks_list=chunks_list, k=3):
    # Measure classification time
    start_classification = time.time()
    classification = classify_english_question_using_gemma(query)
    classification_time = time.time() - start_classification

    if type == "easy":
        decomposition_time = 0.0
        start_retrieval = time.time()
        context = retrieve_documents(query, k)
        retrieval_time = time.time() - start_retrieval
        return context, classification, classification_time, decomposition_time, retrieval_time

    else:
        start_decomposition = time.time()
        decomposition = decompose_english_query_using_gemma(query)
        q1 = decomposition.get("q1", "")
        q2 = decomposition.get("q2", "")
        decomposition_time = time.time() - start_decomposition

        start_retrieval = time.time()
        main_context = retrieve_documents(q1, k)
        additional_contexts = []

        with ThreadPoolExecutor() as executor:
            futures = [
                executor.submit(expand_multihop_context, ctx, query, q1, q2, k)
                for ctx in main_context[:k]
            ]

            for future in as_completed(futures):
                result = future.result()
                additional_contexts.extend(result)

        main_context.extend(additional_contexts)
        retrieval_time = time.time() - start_retrieval

        return main_context, classification, classification_time, decomposition_time, retrieval_time


In [18]:
import time

def multihop_handling_LQR(query, type, model=model, index=index, chunks_list=chunks_list, k=3):
    # Step 1: Get context and timings
    context, classification, classification_time, decomposition_time, retrieval_time = get_context_of_multihop(
        query, type, model=model, index=index, chunks_list=chunks_list, k=k
    )

    # Flatten context if it's a list of strings
    if isinstance(context, list):
        combined_context = "\n".join(context)
    else:
        combined_context = context

    # Step 2: Generate answer and measure time
    start_gen = time.time()
    final_answer = generate_using_llama3(combined_context,query)
    generation_time = time.time() - start_gen

    # Step 3: Compute total time
    total_time = classification_time + decomposition_time + retrieval_time + generation_time

    return {
        "classification": classification,
        "retrieved_context": context,
        "final_answer": final_answer,
        "timings": {
            "classification_time": classification_time,
            "decomposition_time": decomposition_time,
            "retrieval_time": retrieval_time,
            "generation_time": generation_time,
            "total_time": total_time
        }
    }


In [19]:
import pandas as pd
from tqdm import tqdm
import time

# Load your source CSV
df = pd.read_csv("../../../Dataset_code_csvs/hotpotQA/hotpotQA_dataset_versions/5884paras_598queries/English/598_QnAs.csv")

# Output CSV path
output_path = "../../results/pipeline results/5884paras_598qna/llama3.1_gemma_res/LQR_processed_results_en.csv"

# Initialize variables
results = []
batch_times = []
total_start = time.time()

# Loop over each question in the DataFrame
for idx, row in tqdm(df.iterrows(), total=len(df)):
    query = row["question"]
    answer = row["answer"]
    question_type = row["level"]

    start_time = time.time()

    try:
        # Run the pipeline
        result = multihop_handling_LQR(query, question_type)

        classification = result["classification"]
        retrieved_context = result["retrieved_context"]
        final_answer = result["final_answer"]
        timings = result["timings"]
        total_time_one = timings["total_time"]

    except Exception as e:
        print(f"Error processing query {idx}: {e}")
        classification = "Error"
        retrieved_context = "Error"
        final_answer = "Error"
        timings = {
            "classification_time": 0,
            "decomposition_time": 0,
            "retrieval_time": 0,
            "generation_time": 0,
            "total_time": 0
        }
        total_time_one = 0

    elapsed = time.time() - start_time
    batch_times.append(elapsed)

    results.append({
        "question": query,
        "answer": answer,
        "classification": classification,
        "retrieved_context": retrieved_context,
        "final_answer": final_answer,
        "classification_time": timings["classification_time"],
        "decomposition_time": timings["decomposition_time"],
        "retrieval_time": timings["retrieval_time"],
        "generation_time": timings["generation_time"],
        "total_time": timings["total_time"],
        "level": question_type
    })

    print(f"Processed record {idx+1}/{len(df)} in {elapsed:.2f} seconds.")

    # Save and report every 100 queries
    if (idx + 1) % 100 == 0:
        pd.DataFrame(results).to_csv(output_path, mode='a', header=not bool(idx), index=False, encoding="utf-8-sig")
        avg_batch_time = sum(batch_times) / len(batch_times)
        print(f"\n--- Saved batch up to record {idx+1}")
        print(f"Average time for last 100 records: {avg_batch_time:.2f} seconds\n")
        results = []
        batch_times = []

# Save any remaining results at the end
if results:
    pd.DataFrame(results).to_csv(output_path, mode='a', header=not bool(len(df) % 100), index=False, encoding="utf-8-sig")

# Final stats
total_elapsed = time.time() - total_start
avg_total_time = total_elapsed / len(df)
print(f"\n✅ All records processed.")
print(f"Total processing time: {total_elapsed:.2f} seconds.")
print(f"Average time per record: {avg_total_time:.2f} seconds.")


  0%|          | 1/598 [00:36<6:04:08, 36.60s/it]

Processed record 1/598 in 36.59 seconds.


  0%|          | 1/598 [01:01<10:14:36, 61.77s/it]


KeyboardInterrupt: 